## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.formula.api import ols

%matplotlib inline

# Data cleaning

In [3]:
data = pd.read_csv('data/kc_house_data.csv')
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [4]:
data.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [5]:
data.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront        object
view              object
condition         object
grade             object
sqft_above         int64
sqft_basement     object
yr_built           int64
yr_renovated     float64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [112]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,sqft_living15,sqft_lot15,salesTax,propertyTaxPerYear,livingAreaRatio,houseAge,pricePerSqft,year_sold
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,NO,NONE,...,0.0,98178,1340,5650,2241.19,2241.19,0.208850,67,39.274336,2014
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,1991.0,98125,1690,7639,5433.80,5433.80,0.354874,71,74.288870,2014
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,0.0,98028,2720,8062,1818.00,1818.00,0.077000,89,18.000000,2015
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,0.0,98136,1360,5000,6100.40,6100.40,0.392000,57,120.800000,2014
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,0.0,98074,1800,7503,5151.00,5151.00,0.207921,35,63.118812,2015


In [109]:
data['view'].head()

0    NONE
1    NONE
2    NONE
3    NONE
4    NONE
Name: view, dtype: object

In [110]:
data['waterfront'].head()

0    NO
1    NO
2    NO
3    NO
4    NO
Name: waterfront, dtype: object

In [111]:
data['yr_renovated'].head()

0       0.0
1    1991.0
2       0.0
3       0.0
4       0.0
Name: yr_renovated, dtype: float64

In [10]:
data['view'] = data['view'].fillna('None')

In [11]:
data['waterfront'] = data['waterfront'].fillna('NO')

In [12]:
data['yr_renovated'] = data['yr_renovated'].fillna(0)

In [49]:
data.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [14]:
data.corr()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,1.000000,-0.016772,0.001150,0.005162,-0.012241,-0.131911,0.018608,-0.010799,0.021617,-0.010612,-0.008211,-0.001798,0.020672,-0.002701,-0.138557
price,-0.016772,1.000000,0.308787,0.525906,0.701917,0.089876,0.256804,0.605368,0.053953,0.117855,-0.053402,0.306692,0.022036,0.585241,0.082845
bedrooms,0.001150,0.308787,1.000000,0.514508,0.578212,0.032471,0.177944,0.479386,0.155670,0.017900,-0.154092,-0.009951,0.132054,0.393406,0.030690
bathrooms,0.005162,0.525906,0.514508,1.000000,0.755758,0.088373,0.502582,0.686668,0.507173,0.047177,-0.204786,0.024280,0.224903,0.569884,0.088303
sqft_living,-0.012241,0.701917,0.578212,0.755758,1.000000,0.173453,0.353953,0.876448,0.318152,0.051060,-0.199802,0.052155,0.241214,0.756402,0.184342
sqft_lot,-0.131911,0.089876,0.032471,0.088373,0.173453,1.000000,-0.004814,0.184139,0.052946,0.004979,-0.129586,-0.085514,0.230227,0.144763,0.718204
floors,0.018608,0.256804,0.177944,0.502582,0.353953,-0.004814,1.000000,0.523989,0.489193,0.003793,-0.059541,0.049239,0.125943,0.280102,-0.010722
sqft_above,-0.010799,0.605368,0.479386,0.686668,0.876448,0.184139,0.523989,1.000000,0.424037,0.020645,-0.261570,-0.001199,0.344842,0.731767,0.195077
yr_built,0.021617,0.053953,0.155670,0.507173,0.318152,0.052946,0.489193,0.424037,1.000000,-0.202555,-0.347210,-0.148370,0.409993,0.326377,0.070777
yr_renovated,-0.010612,0.117855,0.017900,0.047177,0.051060,0.004979,0.003793,0.020645,-0.202555,1.000000,0.062397,0.027970,-0.064543,0.000683,0.004286


In [75]:
data['zipcode'].value_counts().head()

98103    602
98038    589
98115    583
98052    574
98117    553
Name: zipcode, dtype: int64

In [43]:
data[['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']].head()

,sqft_living,sqft_lot,sqft_above,sqft_basement,sqft_living15,sqft_lot15
0,1180,5650,1180,0.0,1340,5650
1,2570,7242,2170,400.0,1690,7639
2,770,10000,770,0.0,2720,8062
3,1960,5000,1050,910.0,1360,5000
4,1680,8080,1680,0.0,1800,7503


In [46]:
data['floors'].value_counts()

1.0    10673
2.0     8235
1.5     1910
3.0      611
2.5      161
3.5        7
Name: floors, dtype: int64

In [48]:
data.drop(columns = 'lat', inplace = True)

In [50]:
data.drop(columns = 'long', inplace = True)

In [51]:
data['salesTax'] = data['price'] * 0.0101

In [54]:
data[['salesTax','price']].head()

,salesTax,price
0,2241.19,221900.0
1,5433.80,538000.0
2,1818.00,180000.0
3,6100.40,604000.0
4,5151.00,510000.0


In [55]:
data['propertyTaxPerYear'] = data['price'] * 0.0101

In [56]:
data['livingAreaRatio'] = data['sqft_living'] / data['sqft_lot']

In [69]:
data['livingAreaRatio'].head()

0    0.208850
1    0.354874
2    0.077000
3    0.392000
4    0.207921
Name: livingAreaRatio, dtype: float64

In [57]:
data['houseAge'] = 2022 - data['yr_built']

In [58]:
data['pricePerSqft'] = data['price'] / data['sqft_lot']

In [85]:
data['date'].head()

0    10/13/2014
1     12/9/2014
2     2/25/2015
3     12/9/2014
4     2/18/2015
Name: date, dtype: object

In [87]:
data['date'] = pd.to_datetime(data['date'])

In [88]:
data['year_sold'] = data['date'].dt.year

In [113]:
data['year_sold'].head()

0    2014
1    2014
2    2015
3    2014
4    2015
Name: year_sold, dtype: int64

In [74]:
data[['price','floors']].corr()

,price,floors
price,1.000000,0.256804
floors,0.256804,1.000000


# If you are builders

### looking for great land lot rebuild to mixed use house

find high demand zip code. find big lot with small house in the area. find house house 

In [107]:
data['zipcode'].value_counts().head(9)

98103    602
98038    589
98115    583
98052    574
98117    553
98042    547
98034    545
98118    507
98023    499
98006    498
Name: zipcode, dtype: int64

In [118]:
data.corr()['pricePerSqft']

id                    0.059813
price                 0.308066
bedrooms             -0.062884
bathrooms             0.202578
sqft_living           0.022404
sqft_lot             -0.206256
floors                0.477914
sqft_above           -0.004770
yr_built              0.124420
yr_renovated          0.034789
zipcode               0.221420
sqft_living15        -0.044397
sqft_lot15           -0.228634
salesTax              0.308066
propertyTaxPerYear    0.308066
livingAreaRatio       0.896127
houseAge             -0.124420
pricePerSqft          1.000000
year_sold             0.010053
Name: pricePerSqft, dtype: float64

In [128]:
data.corr()['sqft_living']

id                   -0.012241
price                 0.701917
bedrooms              0.578212
bathrooms             0.755758
sqft_living           1.000000
sqft_lot              0.173453
floors                0.353953
sqft_above            0.876448
yr_built              0.318152
yr_renovated          0.051060
zipcode              -0.199802
sqft_living15         0.756402
sqft_lot15            0.184342
salesTax              0.701917
propertyTaxPerYear    0.701917
livingAreaRatio       0.076988
houseAge             -0.318152
pricePerSqft          0.022404
year_sold            -0.029014
Name: sqft_living, dtype: float64

In [122]:
simple_formula = 'pricePerSqft~price'
simple_model = ols(simple_formula, data).fit()
simple_model_summary = simple_model.summary()
simple_model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           pricePerSqft   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.095
Method:                 Least Squares   F-statistic:                     2264.
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        10:42:06   Log-Likelihood:            -1.2770e+05
No. Observations:               21597   AIC:                         2.554e+05
Df Residuals:                   21595   BIC:                         2.554e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     46.3725      1.083     42.819      0.000      44.250      48.495
price       7.888e-05   1.66e-06     47.585      0.000    7.56e-05    8.21e-05
==============================================================================
Omnibus:                    16212.990   Durbin-Watson:                   1.749
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           489434.062
Skew:                           3.325   Prob(JB):                         0.00
Kurtosis:                      25.354   Cond. No.                     1.16e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.16e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [125]:
multiple_formula = 'pricePerSqft~price+livingAreaRatio'
multiple_model = ols(multiple_formula, data).fit()
multiple_model_summary = multiple_model.summary()
multiple_model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           pricePerSqft   R-squared:                       0.843
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                 5.787e+04
Date:                Tue, 29 Mar 2022   Prob (F-statistic):               0.00
Time:                        10:48:22   Log-Likelihood:            -1.0880e+05
No. Observations:               21597   AIC:                         2.176e+05
Df Residuals:                   21594   BIC:                         2.176e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         -37.6399      0.522    -72.108      0.000     -38.663     -36.617
price            5.142e-05   6.96e-07     73.858      0.000    5.01e-05    5.28e-05
livingAreaRatio   305.3147      0.953    320.480      0.000     303.447     307.182
==============================================================================
Omnibus:                    10329.168   Durbin-Watson:                   1.971
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           334462.306
Skew:                           1.680   Prob(JB):                         0.00
Kurtosis:                      21.984   Cond. No.                     2.57e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.57e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [131]:
data['livingAreaRatio'].mean()

0.32375545559503777

In [140]:
df = data[['price','sqft_living', 'sqft_lot', 'floors', 'sqft_above','livingAreaRatio','houseAge','pricePerSqft','zipcode']]
df

,price,sqft_living,sqft_lot,floors,sqft_above,livingAreaRatio,houseAge,pricePerSqft,zipcode
0,221900.0,1180,5650,1.0,1180,0.208850,67,39.274336,98178
1,538000.0,2570,7242,2.0,2170,0.354874,71,74.288870,98125
2,180000.0,770,10000,1.0,770,0.077000,89,18.000000,98028
3,604000.0,1960,5000,1.0,1050,0.392000,57,120.800000,98136
4,510000.0,1680,8080,1.0,1680,0.207921,35,63.118812,98074
...,...,...,...,...,...,...,...,...,...
21592,360000.0,1530,1131,3.0,1530,1.352785,13,318.302387,98103
21593,400000.0,2310,5813,2.0,2310,0.397385,8,68.811285,98146
21594,402101.0,1020,1350,2.0,1020,0.755556,13,297.852593,98144
21595,400000.0,1600,2388,2.0,1600,0.670017,18,167.504188,98027


In [141]:
zipcode_df = df.groupby(by = ["zipcode"]).mean()

In [147]:
zipcode_df.sort_values(by = ['pricePerSqft'], ascending = False).head()

,price,sqft_living,sqft_lot,floors,sqft_above,livingAreaRatio,houseAge,pricePerSqft
zipcode,,,,,,,,
98102,8.996077e+05,2151.048077,3605.153846,1.990385,1752.442308,0.777736,69.836538,322.561090
98119,8.497148e+05,2005.614130,3615.217391,1.793478,1589.961957,0.692205,74.380435,287.432530
98109,8.800778e+05,2054.779817,3621.082569,1.729358,1637.348624,0.671971,77.972477,281.081375
98112,1.096239e+06,2498.743494,4990.431227,1.806691,2005.695167,0.621954,77.550186,261.968320
98107,5.791098e+05,1579.548872,3194.368421,1.834586,1317.218045,0.660090,62.368421,243.382314


In [ ]:
df_98102 = 